# Helper

In [5]:
import pickle
import spacy
import json
import numpy as np

from keras.models import load_model
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model, load_model

In [6]:
class Embedder(object):
    def __init__(self, max_length=100):
        self.max_length = max_length
        print("Loading spacy...")
        self.nlp = spacy.load('en')
        print("...done")

    def embed(self, text):
        return self._pad(self._vectors(text))

    def shape(self):
        return (self.max_length, 300)

    def _vectors(self, text):
        doc = self.nlp(text)
        vectors = []
        for token in doc:
            vectors.append(token.vector)
        return vectors

    def _pad(self, vectors):
        vector_dim = len(vectors[0])
        sequence = np.zeros((self.max_length, vector_dim))
        for i, vector in enumerate(vectors):
            if i == self.max_length:
                break
            sequence[i] = vector
        return sequence

class Predictor(object):
    def __init__(self, embedder, model_path, prompt_names_path):
        self.model = load_model(model_path)
        self.prompt_names = pickle.load(open(prompt_names_path, "rb"))
        self.embedder = embedder

    def predict(self, text):
        proba = self._predict_proba(text)
        return self.prompt_names[np.argmax(proba)]

    def _predict_proba(self, text):
        embedding = self.embedder.embed(text)
        return self.model.predict(np.array([embedding]))

def predict(model, text):
    embedding = embedder.embed(text)
    return model.predict(np.array([embedding]))


def translate(prediction, prompt_names):
    return prompt_names[np.argmax(prediction)]


def suggest(text):
    prediction = predict(loaded_model, text)
    return translate(prediction, loaded_prompt_names)    

# Prepare Data

In [27]:
def load_data(data_dir, limit=0):
    examples = []
    prompt_names = {}
    with open(data_dir + '/01.json') as data_file:
        data = json.load(data_file)
    for i, prompt in enumerate(data):
        prompt_names[i] = prompt["prompt"]
        for message in prompt["response"]:
            examples.append((message, i))
    np.random.shuffle(examples)
    #if limit >= 1:
    #    examples = examples[:limit]
    messages, prompts = zip(*examples)
    return examples, prompt_names

def dummy_encode(array, num_classes=None):
    array = np.array(array)
    if num_classes is None:
        num_classes = max(array) + 1
    result = np.zeros((len(array), num_classes))
    result[np.arange(len(array)), array] = 1
    return result

def create_dataset(type):
    if type == 'train':
        data_dir = 'data/train'
    elif type == 'dev':
        data_dir = 'data/dev'
    examples, prompt_names = load_data(data_dir)
    X = []
    y = []
    for example in examples:
        message, prompt = example[0], example[1]
        X.append(embedder.embed(message))
        y.append(prompt)
    return np.array(X), dummy_encode(np.array(y)), prompt_names

In [28]:
embedder = Embedder()
X_train, y_train, prompt_names_train = create_dataset('train')
X_dev, y_dev, prompt_names_dev = create_dataset('dev')
X, y, prompt_names = X_train, y_train, prompt_names_train

Loading spacy...
...done


# Create CNN

In [34]:
def build_model(input_shape):
    sequences = Input(shape=input_shape)
    x = Conv1D(400, 5, activation='relu')(sequences)
    x = MaxPooling1D(2)(x)
    x = Conv1D(400, 3, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dense(400, activation='relu')(x)
    preds = Dense(len(prompt_names), activation='softmax')(x)
    model = Model(input=sequences, output=preds)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

def build_model2(input_shape):
    sequences = Input(shape=input_shape)
    x = Conv1D(512, 5, activation='relu')(sequences)
    x = MaxPooling1D(2)(x)
    x = Conv1D(512, 3, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    preds = Dense(len(prompt_names), activation='softmax')(x)
    model = Model(input=sequences, output=preds)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

def build_model3(input_shape):
    sequences = Input(shape=input_shape)
    x = Conv1D(400, 5, activation='relu')(sequences)
    x = MaxPooling1D(2)(x)
    x = Conv1D(400, 2, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dense(400, activation='relu')(x)
    preds = Dense(len(prompt_names), activation='softmax')(x)
    model = Model(input=sequences, output=preds)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

def save_model(model, model_name):
    model.save('output/' + model_name)

# Prompts

In [30]:
pickle.dump(prompt_names, open("output/prompt_names.p", "wb"))
loaded_prompt_names = pickle.load(open("output/prompt_names.p", "rb"))

# Model 1

In [70]:
model = build_model((X.shape[1], X.shape[2]))
history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), nb_epoch=40, batch_size=256)
save_model(model, 'mymodel')
embedder = Embedder()

Train on 26408 samples, validate on 8794 samples
Epoch 1/40
26408/26408 [==============================] - 11s - loss: 4.5642 - acc: 0.0526 - val_loss: 3.8094 - val_acc: 0.1071
Epoch 2/40
26408/26408 [==============================] - 10s - loss: 3.0429 - acc: 0.1893 - val_loss: 2.9008 - val_acc: 0.2319
Epoch 3/40
26408/26408 [==============================] - 10s - loss: 2.1049 - acc: 0.3804 - val_loss: 1.7825 - val_acc: 0.4775
Epoch 4/40
26408/26408 [==============================] - 10s - loss: 1.4156 - acc: 0.5802 - val_loss: 1.2680 - val_acc: 0.6257
Epoch 5/40
26408/26408 [==============================] - 10s - loss: 1.0044 - acc: 0.7371 - val_loss: 0.7410 - val_acc: 0.8410
Epoch 6/40
26408/26408 [==============================] - 10s - loss: 0.7395 - acc: 0.8260 - val_loss: 0.5572 - val_acc: 0.8780
Epoch 7/40
26408/26408 [==============================] - 10s - loss: 0.6071 - acc: 0.8635 - val_loss: 0.4789 - val_acc: 0.8947
Epoch 8/40
26408/26408 [==============================]

In [72]:
loaded_model = load_model('output/mymodel')
predictor1 = Predictor(embedder, 'output/mymodel', 'output/prompt_names.p')

In [77]:
print(predictor1.predict('i want to pay with by credit card'),
predictor1.predict('can i buy by post'),
predictor1.predict('i would pay by dollars'),
predictor1.predict('can i am a pay with credit card'))

Sag: Ich möchte mit Kreditkarte bezahlen Sag: Ich möchte mit Visa bezahlen Sag: Ich möchte mit Dollars bezahlen Sag: Ich möchte mit Kreditkarte bezahlen


# Model 2

In [78]:
modelF = build_model((X.shape[1], X.shape[2]))
history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), nb_epoch=100, batch_size=256)
save_model(model, 'mymodelF')
loaded_model = load_model('output/mymodelF')
predictor2 = Predictor(embedder, 'output/mymodelF', 'output/prompt_names.p')

Train on 26408 samples, validate on 8794 samples
Epoch 1/100
26408/26408 [==============================] - 10s - loss: 0.0693 - acc: 0.9817 - val_loss: 0.1000 - val_acc: 0.9777
Epoch 2/100
26408/26408 [==============================] - 10s - loss: 0.0721 - acc: 0.9810 - val_loss: 0.0965 - val_acc: 0.9765
Epoch 3/100
26408/26408 [==============================] - 10s - loss: 0.0762 - acc: 0.9808 - val_loss: 0.1070 - val_acc: 0.9784
Epoch 4/100
26408/26408 [==============================] - 10s - loss: 0.0605 - acc: 0.9824 - val_loss: 0.1020 - val_acc: 0.9766
Epoch 5/100
26408/26408 [==============================] - 10s - loss: 0.0927 - acc: 0.9784 - val_loss: 0.1097 - val_acc: 0.9724
Epoch 6/100
26408/26408 [==============================] - 10s - loss: 0.0506 - acc: 0.9854 - val_loss: 0.1097 - val_acc: 0.9752
Epoch 7/100
26408/26408 [==============================] - 10s - loss: 0.0527 - acc: 0.9846 - val_loss: 0.4750 - val_acc: 0.9182
Epoch 8/100
26408/26408 [=======================

In [ ]:
print(predictor2.predict('i want to pay with by credit card'),
predictor2.predict('can i buy by post'),
predictor2.predict('i would pay by dollars'),
predictor2.predict('can i am a pay with credit card'))

# Model 3

In [35]:
model3 = build_model3((X.shape[1], X.shape[2]))
history = model3.fit(X_train, y_train, validation_data=(X_dev, y_dev), nb_epoch=100, batch_size=256)
save_model(model3, 'model3')
loaded_model = load_model('output/model3')


Train on 26408 samples, validate on 8794 samples
Epoch 1/100
26408/26408 [==============================] - 11s - loss: 4.6290 - acc: 0.0504 - val_loss: 4.1760 - val_acc: 0.0744
Epoch 2/100
26408/26408 [==============================] - 9s - loss: 3.0795 - acc: 0.1940 - val_loss: 2.8512 - val_acc: 0.2453
Epoch 3/100
26408/26408 [==============================] - 9s - loss: 2.0941 - acc: 0.4026 - val_loss: 2.2833 - val_acc: 0.3550
Epoch 4/100
26408/26408 [==============================] - 9s - loss: 1.3803 - acc: 0.6174 - val_loss: 1.5603 - val_acc: 0.5392
Epoch 5/100
26408/26408 [==============================] - 9s - loss: 0.9501 - acc: 0.7576 - val_loss: 0.7155 - val_acc: 0.8407
Epoch 6/100
26408/26408 [==============================] - 9s - loss: 0.7186 - acc: 0.8299 - val_loss: 0.5472 - val_acc: 0.8858
Epoch 7/100
26408/26408 [==============================] - 9s - loss: 0.5509 - acc: 0.8783 - val_loss: 0.4539 - val_acc: 0.9055
Epoch 8/100
26408/26408 [=============================

In [ ]:
predictor3 = Predictor(embedder, 'output/model3', 'output/prompt_names.p')
print(predictor3.predict('i want to pay with by credit card'),
predictor3.predict('can i buy by post'),
predictor3.predict('i would pay by dollars'),
predictor3.predict('can i am a pay with credit card'))

# Model 4

In [31]:
model4 = build_model2((X.shape[1], X.shape[2]))
history = model4.fit(X_train, y_train, validation_data=(X_dev, y_dev), nb_epoch=100, batch_size=256)
save_model(model4, 'model4')
loaded_model = load_model('output/model4')
predictor4 = Predictor(embedder, 'output/model4', 'output/prompt_names.p')

Train on 26408 samples, validate on 8794 samples
Epoch 1/100
26408/26408 [==============================] - 12s - loss: 4.5833 - acc: 0.0527 - val_loss: 3.7654 - val_acc: 0.0930
Epoch 2/100
26408/26408 [==============================] - 11s - loss: 2.8122 - acc: 0.2356 - val_loss: 2.5385 - val_acc: 0.3073
Epoch 3/100
26408/26408 [==============================] - 11s - loss: 1.7698 - acc: 0.4830 - val_loss: 1.5437 - val_acc: 0.5889
Epoch 4/100
26408/26408 [==============================] - 12s - loss: 1.0456 - acc: 0.7327 - val_loss: 1.3984 - val_acc: 0.6491
Epoch 5/100
26408/26408 [==============================] - 12s - loss: 0.7663 - acc: 0.8223 - val_loss: 1.1758 - val_acc: 0.6674
Epoch 6/100
26408/26408 [==============================] - 12s - loss: 0.5661 - acc: 0.8720 - val_loss: 1.2474 - val_acc: 0.6301
Epoch 7/100
26408/26408 [==============================] - 11s - loss: 0.4397 - acc: 0.9048 - val_loss: 0.3479 - val_acc: 0.9318
Epoch 8/100
26408/26408 [=======================

In [32]:
print(predictor4.predict('i want to pay with by credit card'),
predictor4.predict('can i buy by post'),
predictor4.predict('i would pay by dollars'),
predictor4.predict('can i am a pay with credit card'))

Sag: Ich möchte mit Kreditkarte bezahlen Sag: Ich möchte mit Visa bezahlen Sag: Ich möchte mit Dollars bezahlen Sag: Ich möchte mit Kreditkarte bezahlen


# Use Model on ST Data

In [84]:
kaldi = open('data/st_data/kaldi_input', 'r')
out = open('output/predicted_meaning', 'a')
for line in kaldi:
    out.write(predictor.predict(line) + '\n')
    
kaldi.close()
out.close()